<a href="https://colab.research.google.com/github/EddyDavies/Trade_With_Twitter/blob/main/ipynb/Transformer_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check GPU

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Sep 11 19:21:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Check TPU

Check Memory

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!git clone https://github.com/EddyDavies/Trade_With_Twitter.git

Cloning into 'Trade_With_Twitter'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 262 (delta 148), reused 186 (delta 81), pack-reused 0
Receiving objects: 100% (262/262), 68.36 KiB | 1.85 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [4]:
!ls ../

bin	 datalab  home	 lib64	opt	    root  srv		     tmp    var
boot	 dev	  lib	 media	proc	    run   sys		     tools
content  etc	  lib32  mnt	python-apt  sbin  tensorflow-1.15.2  usr


In [5]:
%cd Trade_With_Twitter/src
!ls

/content/Trade_With_Twitter/src
bert_farm.ipynb  decorators.py	sentiment  utils.py
bert_farm.py	 prices		twitter


In [6]:
try:
    from google.colab import drive
    IN_COLAB = True
    drive.mount('/content/drive')
except:
    IN_COLAB = False

Mounted at /content/drive


In [7]:
!pip install -r ../requirements.txt

     |████████████████████████████████| 2.8 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 77.1 MB/s 
     |████████████████████████████████| 50 kB 9.3 MB/s 
     |████████████████████████████████| 3.3 MB 82.3 MB/s 
     |████████████████████████████████| 636 kB 87.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
import os
import pandas as pd
from tqdm import tqdm

from utils import get_date_array, get_month_array
from sentiment.inference import to_dict_of_lists, get_paths

if IN_COLAB:
    source, results_folder = get_paths(data_folder='../../drive/MyDrive/ColabData')
else:
    source, results_folder = get_paths()

In [9]:
dates_range = ("2017-01-01", "2021-06-20")
months_range = ["Jan 17", "Jun 21"]

months = get_month_array(months_range)
dates = get_date_array(dates_range)

In [10]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [13]:
from sentiment.inference import get_tweets, get_sentiments, save_sentiments, check_last_day

for date in dates:
    ids, tweets = get_tweets(date, source)

    percentage_per_chunk = 5
    save_every = 2000
    results = get_sentiments(date, tweets, sentiment_analysis, results_folder,
        save_every=save_every, percentage_per_chunk=percentage_per_chunk)

    outputs = to_dict_of_lists(results)
    outputs["ids"] = ids

    check_last_day(results, results_folder, date)

TypeError: ignored

In [ ]:
!ls ../

In [ ]:
# !rm -r ../data/bitcoin_scores

In [ ]:
print(results_folder)

In [ ]:
!git pull